In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pyarrow.parquet as pq

parquet_path = "../data/frustration/v3_frustration.parquet.gzip"
t_frust = -1
t_non_frust = 0.55

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pq.read_table(parquet_path).to_pandas()
df.head(2)

,proteinID,full_seq,res_seq,res_idx,frst_idx,frst_class,set,cath_T_id
0,AF-A0A009EQP3-F1-model_v4_TED02,MKESLRLRLDQLSDRHEELTALLADVEVISDNKRFRQLSREHNDLT...,"[Y, L, E, I, R, A, G, T, G, G, D, E, A, A, I, ...","[111, 112, 113, 114, 115, 116, 117, 118, 119, ...","[0.235, 1.632, -0.844, 1.365, 0.282, -0.384, 0...","[9, 6, 11, 6, 9, 10, 9, 9, 9, 11, 12, 13, 9, 8...",train,3.30.70
1,AF-A0A009F754-F1-model_v4_TED02,MANPAQLVRHKLLNTFFSRHSVWFACITIAVIFTIFHIGYEPRYIY...,"[R, I, L, I, G, N, E, Q, C, T, Q, P, Y, S, A, ...","[164, 165, 166, 167, 168, 169, 170, 171, 172, ...","[-1.068, 1.469, -0.317, 1.098, -0.663, -0.771,...","[12, 6, 10, 7, 11, 11, 7, 10, 3, 9, 13, 11, 9,...",train,3.20.20


In [15]:
df_sub = df.sample(1000, random_state=42).copy()

In [13]:
pd.cut(df['frst_idx'].iloc[0], bins=[-np.inf, t_frust, t_non_frust, np.inf], labels=[0, 1, 2])

[1, 2, 1, 2, 1, ..., 1, 2, 2, 0, 1]
Length: 163
Categories (3, int64): [0 < 1 < 2]

In [35]:
frst_class_3 = []
df['frst_idx'].apply(lambda x: frst_class_3.append(list(pd.cut(x, bins=[-np.inf, t_frust, t_non_frust, np.inf], labels=[0, 1, 2]))))
df['frst_class_3'] = frst_class_3

In [36]:
distr = np.unique(np.concatenate(df_sub['frst_class_3'].values), return_counts=True)[1]
(distr / distr.sum()).round(3)

array([0.117, 0.504, 0.379])

In [37]:
df.to_parquet("../data/frustration/v4_frustration.parquet.gzip", compression="gzip", index=False)

In [38]:
df = pq.read_table("../data/frustration/v4_frustration.parquet.gzip").to_pandas()
df.head(2)

,proteinID,full_seq,res_seq,res_idx,frst_idx,frst_class,set,cath_T_id,frst_class_3
0,AF-A0A009EQP3-F1-model_v4_TED02,MKESLRLRLDQLSDRHEELTALLADVEVISDNKRFRQLSREHNDLT...,"[Y, L, E, I, R, A, G, T, G, G, D, E, A, A, I, ...","[111, 112, 113, 114, 115, 116, 117, 118, 119, ...","[0.235, 1.632, -0.844, 1.365, 0.282, -0.384, 0...","[9, 6, 11, 6, 9, 10, 9, 9, 9, 11, 12, 13, 9, 8...",train,3.30.70,"[1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, ..."
1,AF-A0A009F754-F1-model_v4_TED02,MANPAQLVRHKLLNTFFSRHSVWFACITIAVIFTIFHIGYEPRYIY...,"[R, I, L, I, G, N, E, Q, C, T, Q, P, Y, S, A, ...","[164, 165, 166, 167, 168, 169, 170, 171, 172, ...","[-1.068, 1.469, -0.317, 1.098, -0.663, -0.771,...","[12, 6, 10, 7, 11, 11, 7, 10, 3, 9, 13, 11, 9,...",train,3.20.20,"[0, 2, 1, 2, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 2, ..."
